In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize

In [2]:
data = pd.read_csv('../Cleaning/Clean_Dataset.csv')

In [3]:
tweets = np.array(data['New_Clean_Text'])[:]
labels = np.array(data['sentiment_word'])[:]

In [4]:
from collections import Counter
Counter(labels)

Counter({'NEU': 36693, 'NEG': 4645, 'POS': 5839})

#### Data Preprocessing

In [5]:
punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

# get rid of punctuation
all_tweets = 'separator'.join(tweets)
all_text = ''.join([c for c in all_tweets if c not in punctuation])

# split by new lines and spaces
tweets_split = all_text.split('separator')
all_text = ' '.join(tweets_split)

# create a list of words
words = all_text.split()

In [6]:
# get rid of web address, twitter id, and digit
new_tweets = []
for tweet in tweets_split:
    tweet = tweet.split()
    new_text = []
    for word in tweet:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_tweets.append(new_text)

#### Encoding the words

In [7]:
#create a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

#use the dictionary to tokenize each tweet
#store the tokenized tweets in tweets_ints
tweets_ints = []
for tweet in new_tweets:
    tweets_ints.append([vocab_to_int[word] for word in tweet])

In [8]:
#stats about vocabulary
print('Unique words: ', len((vocab_to_int)))
print()

#print tokens in first tweet
print('Tokenized Tweet: \n', tweets_ints[0])

Unique words:  46030

Tokenized Tweet: 
 [2983, 4, 865, 87, 1636, 516, 38, 586, 369, 164, 5, 110, 2, 1108, 98, 1510, 1162, 287, 9, 32, 45, 1]


#### Encoding the labels

In [9]:
encoded_labels = []
for label in labels:
    if label == 'NEU':
        encoded_labels.append(1)
    elif label == 'NEG':
        encoded_labels.append(0)
    else:
        encoded_labels.append(1)
        
encoded_labels = np.asarray(encoded_labels)

#### Padding Sequence

In [10]:
def pad_features(tweets_ints, seq_length):
    ''' 
    Return features of tweet_ints, where each review is padded with 0's 
    or truncated to the input seq_length.
    '''
    #get the correct rowe x cols shape
    features = np.zeros((len(tweets_ints), seq_length), dtype=int)
    
    for  i, row in enumerate(tweets_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
        
    return features
    

In [11]:
# Test Implementation
seq_length = 30
features = pad_features(tweets_ints, seq_length=seq_length)

#test statements
assert len(features) == len(tweets_ints), "The features should have as many rows as tweet"
assert len(features[0]) == seq_length, "The features row should contain seq_length values"

print(features[:10,:10])

[[   0    0    0    0    0    0    0    0 2983    4]
 [   0    0    0    0    0    0    0    0    0    0]
 [  86   84   10  957   79    1  179  122   10 2122]
 [1636   12  110  883    9   18 1383  118    5  562]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    1   55 7217 2316]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]


### Training, Validation and Test

In [12]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of the resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(37742, 30) 
Validation set: 	(4718, 30) 
Test set: 		(4718, 30)


In [13]:
dummy_data = np.zeros((1,), dtype=np.int32)

test_y = np.concatenate([test_y,dummy_data], axis=0)

##### DataLoaders and batching

In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

#create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [15]:
#obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) #batch_size, seq_length
print('Sample input : \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) #batch size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 30])
Sample input : 
 tensor([[    0,     0,     0,  ...,     2,     1,     6],
        [    0,     0,     0,  ...,   504,   100,     1],
        [  671,     6,  1527,  ...,   436,     5,   215],
        ...,
        [   12,  5005,    84,  ...,     4,    15,  1640],
        [    0,     0,     0,  ...,     1,    97,    11],
        [    0,     0,     0,  ..., 11632,  6325,   890]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 1], dtype=torch.int32)


#### Sentiment Network with Pytorch

In [16]:
#first check if GPU is available
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU')
else:
    print('No GPU available, training on CPU')

No GPU available, training on CPU


In [17]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

##### Instantiate the Network

In [18]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(46031, 200)
  (lstm): LSTM(200, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)


#### Training

In [19]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [20]:
# training params

epochs = 10 

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: 0.449899... Val Loss: 0.318628
Epoch: 1/10... Step: 200... Loss: 0.334309... Val Loss: 0.315018
Epoch: 1/10... Step: 300... Loss: 0.321880... Val Loss: 0.314570
Epoch: 1/10... Step: 400... Loss: 0.366234... Val Loss: 0.313262
Epoch: 1/10... Step: 500... Loss: 0.283471... Val Loss: 0.314521
Epoch: 1/10... Step: 600... Loss: 0.378582... Val Loss: 0.314729
Epoch: 1/10... Step: 700... Loss: 0.478364... Val Loss: 0.315603
Epoch: 2/10... Step: 800... Loss: 0.352038... Val Loss: 0.316594
Epoch: 2/10... Step: 900... Loss: 0.235676... Val Loss: 0.315958
Epoch: 2/10... Step: 1000... Loss: 0.447152... Val Loss: 0.317976
Epoch: 2/10... Step: 1100... Loss: 0.237796... Val Loss: 0.318060
Epoch: 2/10... Step: 1200... Loss: 0.146242... Val Loss: 0.318538
Epoch: 2/10... Step: 1300... Loss: 0.245143... Val Loss: 0.319057
Epoch: 2/10... Step: 1400... Loss: 0.253591... Val Loss: 0.318209
Epoch: 2/10... Step: 1500... Loss: 0.443034... Val Loss: 0.319170
Epoch: 3/10... Step

#### Testing

In [21]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.874
Test accuracy: 0.836


#### Create a Predict Function

In [22]:
# negative test review
test_review = "Metaverse sangat jelek, sampah"

In [23]:
def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()
    
    # get rid of web address, twitter id, and digit
    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review)
print(test_ints)

[[1, 190, 1047, 1279]]


In [24]:
# test sequence padding
seq_length=30
features = pad_features(test_ints, seq_length)

print(features)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    1  190
  1047 1279]]


In [25]:
#test sequence padding
seq_length = pad_features(test_ints, seq_length)

print(features)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    1  190
  1047 1279]]


In [26]:
#test conversion to tensor and pass into model
feature_tensor = torch.from_numpy(features)
print(feature_tensor.size())

torch.Size([1, 30])


In [27]:
def predict(net, test_review, sequence_length=30):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Neutral Tweet detected.")
    else:
        print("Negative Tweet detected.")
        

In [28]:
seq_length = 30 #good to use the length that was trained on